# Set Up Environment

In [ ]:
from iconservice import AddressPrefix, Address
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder,DepositTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from checkscore.repeater import retry
from shutil import make_archive
import requests
import json
import time
import random
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass
from dotenv import dotenv_values

deployment_status = {}
print(1)

In [ ]:
# cell 3
NETWORK = "mainnet"

connections = {
    "mainnet": {"iconservice": "https://ctz.solidwallet.io", "nid": 1},
    "yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
    "euljiro": {"iconservice": "https://test-ctz.solidwallet.io", "nid": 2},
    "pagoda": {"iconservice": "https://zicon.net.solidwallet.io", "nid": 80},
    "goloop": {"iconservice": "http://18.237.205.52:9082/", "nid": 3},
    "local": {"iconservice": "http://localhost:9000/", "nid": 3},
    "berlin": {"iconservice": "https://berlin.net.solidwallet.io", "nid": 7},
    "lisbon": {"iconservice": "https://lisbon.net.solidwallet.io", "nid": 2},
    "sejong": {"iconservice": "https://sejong.net.solidwallet.io", "nid": 83}
}

env = connections[NETWORK]

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]


@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result


print(f"network -> {NETWORK}")
print(f"nid -> {NID}")

In [ ]:
# cell 4
# initialized all methods
def get_status_and_print(to, method, response, logs = False) -> int:
    """
    get status and print response
    :param to:
    :param method:
    :param response:
    :return: status
    """
    status = response['status']
    print(f'SCORE {to} ::: Method {method} ::: Status: {status}')
    if logs:
        if len(response["eventLogs"]) > 0:
            for item in response["eventLogs"]:
                print(f'{item} \n')
        if status == 0:
            print(f'Failure: {response["failure"]}')
    return status


def _deploy_contract(_contract_name, params):
    deploy_transaction = DeployTransactionBuilder() \
        .from_(deployer_wallet.get_address()) \
        .to(GOVERNANCE_ADDRESS) \
        .nid(NID) \
        .nonce(100) \
        .step_limit(100000000000) \
        .content_type("application/zip") \
        .content(gen_deploy_data_content(_contract_name)) \
        .params(params) \
        .build()
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    pprint(tx_hash)
    res = get_tx_result(tx_hash)
    status = get_status_and_print(_contract_name, "deploy", res)

    return {
        "address": res.get('scoreAddress', ''),
        "status": status
    }


def deploy_contract(_contract_name):
    """
    deploy all contracts excepts oToken and dToken
    :param _contract_name:  name of contract - should match with package name
    :return: SCORE address
    """
    params = {}
    if _contract_name == "workerToken":
        params = {'_initialSupply': int(config.get("worker_token_initial")), '_decimals': 18}
    if _contract_name == "rewardDistribution":
        DISTRIBUTION_PERCENTAGE = [
            {"recipient": "worker", "percentage": f'{WORKER_DIST_PERCENTAGE}'},
            {"recipient": "daoFund", "percentage": f'{DAO_DIST_PERCENTAGE}'},
            {"recipient": "lendingBorrow", "percentage": f'{LENDING_BORROW_PERCENTAGE}'},
            {"recipient": "liquidityProvider", "percentage": f'{LP_OMM_STAKING_PERCENTAGE}'}
        ]
        params = {"_distPercentage": DISTRIBUTION_PERCENTAGE, "_startTimestamp": TIMESTAMP}

    if _contract_name not in ['addressProvider', "workerToken"]:
        params['_addressProvider'] = contracts['addressProvider']

    result = _deploy_contract(_contract_name, params)
    deployment_status[f"deploy_contract_{_contract_name}"] = result["status"]
    return result["address"]

def deposit_transaction(_from, _to, _value, _method="add"):
    """
    Deposit fee to contracts for fee sharing
    :param _from: transaction wallet
    :param _to: contract to deposit fee for fee sharing
    :param _value: amount to deposit
    :param _method: method
    :return: status of transaction
    """
    print(f"Depositing {_value} to {_to} contract for fee sharing")
    transaction = DepositTransactionBuilder() \
        .from_(_from.get_address()) \
        .to(contracts[_to]) \
        .nid(NID) \
        .value(_value) \
        .step_limit(1000000000) \
        .nonce(100) \
        .action(_method) \
        .build()
    
    signed_transaction = SignedTransaction(transaction, _from)
    tx_hash = icon_service.send_transaction(signed_transaction)
    response = get_tx_result(tx_hash)
    status = get_status_and_print(_to, _method, response)
    return status

def deploy_token_contract(token, params):
    """
    deploy  oTokens and dTokens
    :param token: token name
    :param params: parameter
    :return: SCORE address
    """
    name = params["name"]
    symbol = params["symbol"]
    decimals = params["decimals"]
    print(f"deploying {name}::{symbol}::{decimals}")
    params = {"_name": name, "_symbol": symbol, "_decimals": decimals, '_addressProvider': contracts['addressProvider']}

    result = _deploy_contract(token, params)
    deployment_status[f"deploy_{token}_contract_{symbol}"] = result["status"]
    return result["address"]


def send_tx(_to, _method, _params, _value, _wallet, _tx_detail = False) -> int:
    """
    send transaction to _to
    :param _to: destination conract name
    :param _method: method
    :param _params:  parameters
    :param _value:  value
    :param _wallet: transaction wallet
    :return: status of transaction
    """
    print(f'Calling {_method}, with parameters {_params} on the {_to} contract.')
    transaction = CallTransactionBuilder() \
        .from_(_wallet.get_address()) \
        .to(contracts[_to]) \
        .value(_value) \
        .step_limit(1_00_000_000) \
        .nid(NID) \
        .nonce(100) \
        .method(_method) \
        .params(_params) \
        .build()
    signed_transaction = SignedTransaction(transaction, _wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    print(f"{_to} ::> {_method} ::> {tx_hash}")
    response = get_tx_result(tx_hash)
    if _tx_detail:
        pprint(response)
    status = get_status_and_print(_to, _method, response)
    return status

def hexToInt(value):
    if type(value) is str:
        return int(value,0)
    if type(value) is dict:
        res ={}
        for k,v in value.items():
            if(not v.startswith("cx")):
                res[k]=int(v,0)
        return res

def call_tx(contact, method, params):
    call = CallBuilder()\
        .from_('hx91bf040426f226b3bfcd2f0b5967bbb0320525ce')\
        .to(contracts[contact])\
        .method(method)\
        .params(params)\
        .build()
    response = icon_service.call(call)
    return response

def send_icx(_to, _value, _wallet) -> int:
    """
    send transaction to _to
    :param _to: destination address
    :param _value:  value
    :param _wallet: transaction wallet
    :return: status of transaction
    """
    transaction = TransactionBuilder()\
        .from_(_wallet.get_address())\
        .to(_to)\
        .value(_value)\
        .step_limit(1_000_000_000)\
        .nid(NID)\
        .nonce(100)\
        .build()
    signed_transaction = SignedTransaction(transaction, _wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    print(f"{_to} ::> icx ::> {tx_hash}")
    get_tx_result(tx_hash)


In [ ]:
contracts = {
  "sicx": "cx2609b924e33ef00b648a409245c7ea394c467824",
  "staking": "cx43e2eec79eb76293c298f2b17aec06097be606e0",
  "iusdc": "cxae3034235540b924dfcc1b45836c293dcc82bfb7",
  "usds": "cxbb2871f468a3008f80b08fdde5b8b951583acf06",
  "bnUSD": "cx88fd7df7ddff82f7cc735c871dc519838cb235bb",
  "bandOracle": "cxe647e0af68a4661566f5e9861ad4ac854de808a2",
  "dex": "cxa0af3165c08318e988cb30993b3048335b94af6c",
  "addressProvider": "cx6a66130200b4f08c65ef394469404378ab52e5b6",
  "daoFund": "cx48a83a6fbdaa205a060514fd23ad6871c070f896",
  "feeProvider": "cx4f3c2edf730f203b1ef1257d645415652ae8b4fb",
  "governance": "cx8190de91c8831f382dcabdbc87968448380c4838",
  "lendingPoolCore": "cxfb312bbd0a244b9e7bb5794c91f4e4acc41dea94",
  "liquidationManager": "cx533d76093a7b14fdbc3e213c7f987f1b6fea976c",
  "priceOracle": "cx189f03875da766878c68753da7492c080bcc2dbe",
  "rewardDistribution": "cx4f2d730ad969f5c839229de42184c5e47aefef6f",
  "dUSDS": "cx4729aa82ba504deae71e888e77fa85c2f0c60429",
  "dICX": "cxdd660884b8b71daf105483a41f190a808bde9ef0",
  "dIUSDC": "cxb31005e782bcbcb651fb546219899f9439c52b69",
  "oUSDS": "cx1837c2a92340e91b115457b853fe927bd9fbdd5f",
  "oICX": "cx0fb973aaab3a26cc99022ba455e5bdfed1d6f0d9",
  "oIUSDC": "cxa8efd07255884afd5069d6011a66b1fe49610013",
  "delegation": "cx841f29ec6ce98b527d49a275e87d427627f1afe5",
  "lendingPool": "cxcb455f26a2c01c686fa7f30e1e3661642dd53c0d",
  "lendingPoolDataProvider": "cx5f9a6ca11b2b761a469965cedab40ada9e503cb5",
  "ommToken": "cx1a29259a59f463a67bb2ef84398b30ca56b5830a", 
  "workerToken": "cx718eb47bcf1da56a32afa53ad128844adcfc8211",
  "stakedLp": "cx015c7f8884d43519aa2bcf634140bd7328730cb6",
  "OMM/sICX": "cx687fa54d7b8328fc560dc7a68a0d44d7f1091a25",
  "OMM/USDS": "cxd5383585ecd157f0588bbf06062699126680e0f7",
  "OMM/IUSDC": "cx53f4dd2c3243f646b3565ecdab6f2c3b7aa6defb",
  "obnUSD": "cx333b95ba1ab735533dc86f81501514083879cde5",
  "dbnUSD": "cx440d7bf3607246a6d73ac089ee8d87fc62a68c80",
  "dBALN": "cx3dd0f6f2e16da7a60c1b34086bbd7943af7b55f3",
  "dOMM": "cx07e2ae66d804bf8a86d68c9348e83271456994ee",
  "obnUSD": "cx333b95ba1ab735533dc86f81501514083879cde5",
  "oBALN": "cx0b33a9e5ee380b5d76223dd5396a20833bbb84a9",
  "oOMM": "cx0a075eb7bc938b8ec49b1a7e9c83d83dd89f9324"
}

### Import deployer wallet

In [ ]:
# cell 5
import getpass

# keystore_password = getpass.getpass(f'Password for deployer wallet keystore : ')
deployer_wallet = KeyWallet.load(bytes.fromhex('c5769504c2712e1c6798291a8e1ad67d7c7d0bfa4eea9773c391ac8e158f73d6'))
print(deployer_wallet.get_address())
print(icon_service.get_balance(deployer_wallet.get_address())/10**18)

In [ ]:
contractsMap = {}
for k,v in contracts.items():
    contractsMap[v] = k

## 1. Collect data before to verify the emission rates

In [ ]:
er = hexToInt(call_tx('rewardDistribution','getAssetEmission',{}))
emission_rates = {}
for k,v in er.items():
    emission_rates[contractsMap[k]] = v
emission_rates

## 2. Collect the users list 

### 2.1 Depositors/Borrowers


In [ ]:
depositors = []

def get_depositors(index):
    wallets = call_tx('lendingPool','getDepositWallets',{'_index': index})
    depositors.extend(wallets)
    if len(wallets) == 50:
        get_depositors(index + 1)
        
get_depositors(0)
len(depositors)

### 2.2 OMM Stakers

In [ ]:
stakers = []
def get_stakers(start):
    params = {'_start': start, '_end': start + 100}
    wallets = call_tx('ommToken','getStakersList',params)
    stakers.extend(wallets)
    if len(wallets) == 100:
        get_stakers(start + 100)
get_stakers(0)
len(stakers)

### 2.3 oToken transfers

In [ ]:
oTokens = ['oUSDS','oICX']
otoken_transfer_wallets = []
for i in oTokens:
    addr = contracts[i]
    resp = requests.get(f"https://main.tracker.solidwallet.io/v3/contract/txList?addr={addr}&page=1&count=25")
    data = resp.json()['data']
    for d in data:
        _hash = d.get('txHash')
        response = requests.get(f"https://main.tracker.solidwallet.io/v3/transaction/txDetail?txHash={_hash}")
        _data = response.json()['data']
        if _data['status'] == 'Success':
            ds = json.loads(_data.get('dataString'))
            if ds.get("method") == "transfer":
                otoken_transfer_wallets.append(ds.get("params").get("_to"))
len(otoken_transfer_wallets)

### 2.4 LP Stakers (get from external script)

In [ ]:
lp_stakers = []

In [ ]:
users = list(set(stakers + depositors + otoken_transfer_wallets + lp_stakers))
len(users)

In [ ]:
users

In [ ]:
with open('MigratedUsers.json','w') as outfile:
    json.dump(users, outfile)

In [ ]:
detailed_users = {}
detailed_users['depositors'] = depositors
detailed_users['stakers'] = stakers
detailed_users['lp_stakers'] = lp_stakers
detailed_users['oTokenTransfers'] = otoken_transfer_wallets
with open('DetailMigratedUsers.json','w') as outfile:
    json.dump(detailed_users, outfile)

## 3. Check prepDelegations

In [ ]:
beforePreps = call_tx('delegation','computeDelegationPercentages',{})
len(beforePreps)

In [ ]:
with open('BeforePrepDelegations.json','w') as outfile:
    json.dump(beforePreps, outfile)

## 4. Get voting weight

In [ ]:
proposals = call_tx('governance','getProposals',{})
proposal_details = []
for proposal in proposals:
    _id = proposal.get('id')
    _timestamp = proposal.get('vote snapshot')
    _weight = int(call_tx('ommToken','totalStakedBalanceOfAt',{'_timestamp': _timestamp}),0)
    proposal_details.append({
        'id': _id,
        'timestamp': _timestamp,
        'weight': _weight
    })
proposal_details

In [ ]:
with open('Proposal_Weights.json','w') as outfile:
    json.dump(proposal_details, outfile)